In [29]:
import geemap
import ee 
import os
import geopandas as gpd
import json
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
# Get ESA World cover
world_cover_collection = ee.ImageCollection("ESA/WorldCover/v100")

In [18]:
world_cover_image = world_cover_collection.first()
geemap.image_props(world_cover_image).getInfo()

{'IMAGE_DATE': '2020-01-01',
 'Map_class_names': ['Trees',
  'Shrubland',
  'Grassland',
  'Cropland',
  'Built-up',
  'Barren / sparse vegetation',
  'Snow and ice',
  'Open water',
  'Herbaceous wetland',
  'Mangroves',
  'Moss and lichen'],
 'Map_class_palette': ['006400',
  'ffbb22',
  'ffff4c',
  'f096ff',
  'fa0000',
  'b4b4b4',
  'f0f0f0',
  '0064c8',
  '0096a0',
  '00cf75',
  'fae6a0'],
 'Map_class_values': [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100],
 'NODATA_value': '0',
 'NOMINAL_SCALE': 9.276624232772797,
 'copyright': 'ESA WorldCover project 2020 / Contains modified Copernicus Sentinel data (2020) processed by ESA WorldCover consortium',
 'creator': 'ESA WorldCover project',
 'doi': 'NA',
 'institution': 'VITO NV',
 'orbit_type': '',
 'platform': 'Sentinel-2 and Sentinel-1',
 'processing_level': 'L3',
 'processing_mode': 'offline',
 'product_version': 'V1.0.0',
 'production_crs': 'EPSG:4326',
 'production_grid': '3x3 degree tiling grid',
 'references': 'https://esa-world

In [21]:
Map.addLayer(world_cover_image, {}, "First image")
Map

Map(bottom=481732.0, center=[14.55425996948678, 121.02793854940143], controls=(WidgetControl(options=['positio…

In [11]:
# get city boundaries
city_boundary = ee.FeatureCollection('users/saifshabou/dataportal/boundaries/PHL-makati')

In [22]:
Map.centerObject(city_boundary,12)
Map.addLayer(city_boundary,{} ,"City boundary")

In [58]:
# convert geojson boundary into ee feature collection
def city_boundary_geojson_to_ee(city_id):
    # read geojson
    geojson_path = '../data/'+city_id+'.geojson'
    file_path = os.path.abspath(geojson_path)
    # open geojson
    with open(file_path) as f:
        json_data = json.load(f)
    # convert geojson to ee feature collection
    ee_data = geojson_to_ee(json_data)
    
    return ee_data

city_boundary_ee = city_boundary_geojson_to_ee(city_id = 'CRI-San_jose-boundary')

In [63]:
city_id = 'CRI-San_jose-boundary'

# clip raster
clip = world_cover_image.clipToCollection(city_boundary_ee)
# output name 
output_file_name = 'data/world_cover_esa/world_cover_'+ city_id+'.tif'
# export
geemap.ee_export_image(clip, 
                       filename=output_file_name, 
                       region=city_boundary_ee.geometry())

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\UrbanShift\github\cities-urbanshift\baseline-indicators\biodiversity\scripts\data\world_cover_esa\world_cover_CRI-San_jose-boundary.tif


In [60]:
# export clipped geotif

def city_world_cover_clip(world_cover_image, city_id, city_boundary_ee):
    # clip raster
    clip = world_cover_image.clipToCollection(ee_data)
    # output name 
    output_file_name = 'world_cover_'+ city_id+'.tif'
    # export
    geemap.ee_export_image(clip, 
                           filename=output_file_name, 
                           region=ee_data.geometry())

output_file_name = 'world_cover_'+ city_id+'.tif'

geemap.ee_export_image(world_cover_image, 
                       filename=output_file_name, 
                       #scale=90, 
                       region=city_boundary_ee, 
                       file_per_band=False)

Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale, argument 'geometry': Invalid type.
Expected type: Geometry.
Actual type: FeatureCollection.


In [54]:
city_id = 'CRI-San_jose-boundary'
geojson_path = '../data/'+city_id+'.geojson'

In [56]:
file_path = os.path.abspath(geojson_path)

In [38]:
with open(file_path) as f:
    json_data = json.load(f)

In [39]:
json_layer = GeoJSON(data=json_data, name='San Jose boundaries', hover_style={'fillColor': 'red' , 'fillOpacity': 0.5})
Map.add_layer(json_layer)

In [41]:
ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data, {}, "San Jose boundaries ee")

In [46]:
world_cover_city = world_cover_collection \
  .filterBounds(ee_data)

In [51]:
geemap.ee_export_image(world_cover_image, 
                       filename="test wc.tif", 
                       #scale=90, 
                       region=ee_data, 
                       file_per_band=False)

Generating URL ...
An error occurred while downloading.
Image.clipToBoundsAndScale, argument 'geometry': Invalid type.
Expected type: Geometry.
Actual type: FeatureCollection.


In [52]:
clip = world_cover_image.clipToCollection(ee_data)

In [53]:
geemap.ee_export_image(clip, 
                       filename='test.tif', 
                       region=ee_data.geometry())

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Saif.Shabou\OneDrive - World Resources Institute\Documents\UrbanShift\github\cities-urbanshift\baseline-indicators\biodiversity\scripts\test.tif


In [5]:
country = 'PHL'

popASun = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj").filterBounds(city_boundary) 

pop = popASun \
  .filter(ee.Filter.inList('country',[country])) \
  .filter(ee.Filter.inList('year',[2020])) \
  .select('population')

population = popASun \
    .select(['population']) \
    .mean() \
    .reduce(ee.Reducer.sum()) \
    .rename('population')

popChildren = popASun \
    .select(['M_0','M_1','M_5','M_10','M_15','F_0','F_1','F_5','F_10','F_15']) \
    .mean() \
    .reduce(ee.Reducer.sum()) \
    .rename('popChildren')


In [6]:
# get children category
data_popChildren = popASun.select(['M_0','M_1','M_5','M_10','M_15','F_0','F_1','F_5','F_10','F_15'])
city_popChildren_img = data_popChildren.map(lambda image: image.clip(city_boundary))
city_popChildren_img = city_popChildren_img \
                       .mean() \
                       .reduce(ee.Reducer.sum()) \
                       .rename('popChildren')

In [20]:
pop_city = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj")

In [24]:
pop_city = population \
  .filter(ee.Filter.geometry(city_boundary)) \

SyntaxError: unexpected EOF while parsing (Temp/ipykernel_19460/2789389396.py, line 2)

In [22]:
pop_city.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'WorldPop/GP/100m/pop_age_sex_cons_unadj',
 'version': 1633091485639135,
 'properties': {'date_range': [1577836800000, 1609459200000],
  'period': 0,
  'type_name': 'ImageCollection',
  'max_mirrored_version': 1616619208130123,
  'keywords': ['demography', 'population', 'worldpop'],
  'thumb': 'https://mw1.google.com/ges/dd/images/WorldPop_GP_100m_pop_thumb.png',
  'description': '<p>Global high-resolution, contemporary data on human population distributions are\na prerequisite for the accurate measurement of the impacts of population\ngrowth, for monitoring changes, and for planning interventions. The WorldPop\nproject aims to meet these needs through the provision of detailed and open\naccess population distribution datasets built using transparent and\npeer-reviewed approaches.</p><p>Full details on the methods and datasets used in constructing the\ndata, along with open access publications, are provided on the\n<a href="https://www.w

In [7]:
popviz = {
  'min': 0.0,
  'max': 150.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}
Map.addLayer(population,popviz,'Population',False,0.5)
Map.addLayer(popChildren,popviz,'Population - Children (<20)',False,0.5)

In [23]:
Map.addLayer(pop_city,popviz,'pop_city',False,0.5)

In [74]:
# export geotif
out_dir = os.path.join(os.path.expanduser('~'), 'data')
filename = os.path.join(out_dir, 'city_pop_children.tif')

In [84]:
# load administrative boundaries: Makati
boundaries_cities = gpd.read_file('https://storage.googleapis.com/data_portal_exposure/data/administrative_boundaries/mapped/PHL_makati.geojson')


In [108]:
# get bbox
xmin, ymin, xmax, ymax = boundaries_cities.total_bounds
geometry = ee.Geometry.Rectangle([xmin, ymin, xmax, ymax])

In [110]:
geemap.ee_export_image(city_popChildren_img, 
                       filename=filename, 
                       #scale=90, 
                       region=geometry, 
                       file_per_band=False)

Generating URL ...
Please wait ...
An error occurred while downloading.
[Errno 2] No such file or directory: 'C:\\Users\\Saif.Shabou\\data\\city_pop_children.zip'


In [61]:
popChildren = popASun \
    .filterBounds(city_boundary) \
    .select(['M_0','M_1','M_5','M_10','M_15','F_0','F_1','F_5','F_10','F_15']) \
    .mean() \
    .reduce(ee.Reducer.sum()) \
    .rename('popChildren')

In [71]:
city_popChildren_img.getInfo()

{'type': 'Image',
 'bands': [{'id': 'popChildren',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [63]:
popviz = {
  'min': 0.0,
  'max': 150.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}
Map.addLayer(population,popviz,'Population',False,0.5)
Map.addLayer(popChildren,popviz,'Population - Children (<20)',False,0.5)


In [72]:
Map.addLayer(city_popChildren_img,popviz,'city_popChildren_img',False,0.5)

In [6]:
# get city boundaries
city_boundary = ee.FeatureCollection('users/saifshabou/dataportal/boundaries/PHL-makati')

In [9]:
# get population data
pop_age_sex = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj")

visualization = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}

Map.centerObject(city_boundary,9)

Map.addLayer(pop_age_sex, visualization, 'Population')

In [18]:
# filter parameters
year = 2020
country_code = 'PHL'

In [19]:
city_pop_age_sex = pop_age_sex.filter(ee.Filter.bounds(city_boundary)) \
    .filterMetadata('year', 'equals', year) \
    .filterMetadata('country', 'equals', country_code) 

In [21]:
Map.addLayer(city_pop_age_sex, visualization, 'Population city')
Map.centerObject(city_boundary,9)

In [33]:
# process population data
city_pop_dataset = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj").filterBounds(city_boundary) 
data_population = city_pop_dataset.select('M_0') #population
city_population_img = data_population.map(lambda image: image.clip(city_boundary))

In [52]:
city_population_img.getInfo()

{'type': 'ImageCollection',
 'bands': [],
 'id': 'WorldPop/GP/100m/pop_age_sex_cons_unadj',
 'version': 1633091485639135,
 'properties': {'date_range': [1577836800000, 1609459200000],
  'period': 0,
  'type_name': 'ImageCollection',
  'max_mirrored_version': 1616619208130123,
  'keywords': ['demography', 'population', 'worldpop'],
  'thumb': 'https://mw1.google.com/ges/dd/images/WorldPop_GP_100m_pop_thumb.png',
  'description': '<p>Global high-resolution, contemporary data on human population distributions are\na prerequisite for the accurate measurement of the impacts of population\ngrowth, for monitoring changes, and for planning interventions. The WorldPop\nproject aims to meet these needs through the provision of detailed and open\naccess population distribution datasets built using transparent and\npeer-reviewed approaches.</p><p>Full details on the methods and datasets used in constructing the\ndata, along with open access publications, are provided on the\n<a href="https://www.w

In [34]:
# viz
city_pop_vis = {
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
}
# adds image layers to map
Map.addLayer(city_population_img, city_pop_vis, 'city_population_img')

In [42]:
# process population data
city_pop_dataset = ee.ImageCollection("WorldPop/GP/100m/pop_age_sex_cons_unadj").filterBounds(city_boundary) 
data_population_children = city_pop_dataset.select('M_0','M_1','M_5','M_10','M_15','F_0','F_1','F_5','F_10','F_15')\
                           .reduce(ee.Reducer.sum())
city_population_children_img = data_population_children.map(lambda image: image.clip(city_boundary))

AttributeError: 'Image' object has no attribute 'map'

In [ ]:
popChildren = popASun \
    .select(['M_0','M_1','M_5','M_10','M_15','F_0','F_1','F_5','F_10','F_15']) \
    .mean() \
    .reduce(ee.Reducer.sum()) \
    .rename('popChildren')

In [43]:
data_population_children.getInfo()

{'type': 'Image',
 'bands': [{'id': 'M_0_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'M_1_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'M_5_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'M_10_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'M_15_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'F_0_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'F_1_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EP

In [39]:
# adds image layers to map
Map.addLayer(city_population_children_img, city_pop_vis, 'city_population_children_img')

EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.